In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Load data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Preprocess data
X_train = X_train.reshape((60000, 28*28))
X_test = X_test.reshape((10000, 28*28))
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(28*28,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model without regularization
history_no_reg = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Train the model with L2 regularization
from tensorflow.keras import regularizers

model_l2 = Sequential()
model_l2.add(Dense(64, activation='relu', input_shape=(28*28,), kernel_regularizer=regularizers.l2(0.001)))
model_l2.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_l2.add(Dense(10, activation='softmax'))

model_l2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history_l2 = model_l2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


In [ ]:
import matplotlib.pyplot as plt

# Plot the training accuracy
plt.plot(history_no_reg.history['accuracy'], label='No regularization')
plt.plot(history_l2.history['accuracy'], label='L2 regularization')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras import regularizers

# Define the model architecture with L1 regularization
model_l1 = Sequential()
model_l1.add(Dense(64, activation='relu', input_shape=(28*28,), kernel_regularizer=regularizers.l1(0.001)))
model_l1.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)))
model_l1.add(Dense(10, activation='softmax'))

model_l1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with L1 regularization
history_l1 = model_l1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Plot the training accuracy for all models
plt.plot(history_no_reg.history['accuracy'], label='No regularization')
plt.plot(history_l1.history['accuracy'], label='L1 regularization')
plt.plot(history_l2.history['accuracy'], label='L2 regularization')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure()

# Define the number of epochs to plot
n_epochs = 10

# Define the update function for the animation
def update_plot(i):
    plt.clf()
    plt.plot(history_no_reg.history['accuracy'][:i+1], label='No regularization')
    plt.plot(history_l1.history['accuracy'][:i+1], label='L1 regularization')
    plt.plot(history_l2.history['accuracy'][:i+1], label='L2 regularization')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training Accuracy (Epoch {})'.format(i+1))
    plt.ylim([0.5, 1])
    plt.xlim([0, n_epochs])

# Create the animation and save it as a GIF
Writer = animation.writers['pillow']
writer = Writer(fps=5)

anim = animation.FuncAnimation(fig, update_plot, frames=n_epochs, repeat=False)
anim.save('training_accuracy.gif', writer=writer)


In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

# Define the number of epochs to plot
n_epochs = 10

# Create the data for the plot
data = []
for i in range(n_epochs):
    data.append({'epoch': i+1, 'accuracy': history_no_reg.history['accuracy'][i], 'regularization': 'No regularization'})
    data.append({'epoch': i+1, 'accuracy': history_l1.history['accuracy'][i], 'regularization': 'L1 regularization'})
    data.append({'epoch': i+1, 'accuracy': history_l2.history['accuracy'][i], 'regularization': 'L2 regularization'})

# Create a separate trace for each regularization type
traces = []
for reg_type in ['No regularization', 'L1 regularization', 'L2 regularization']:
    reg_data = [d for d in data if d['regularization'] == reg_type]
    trace = go.Scatter(x=[d['epoch'] for d in reg_data],
                       y=[d['accuracy'] for d in reg_data],
                       mode='lines+markers',
                       name=reg_type,
                       line=dict(dash='dashdot' if reg_type == 'L1 regularization' else 'solid')
                      )
    traces.append(trace)

# Create the animated plot
fig = go.Figure(data=traces,
                layout=go.Layout(xaxis=dict(range=[0, n_epochs], title='Epoch'),
                                 yaxis=dict(range=[0.5, 1], title='Accuracy'),
                                 title='Training Accuracy',
                                 updatemenus=[dict(type='buttons',
                                                   showactive=False,
                                                   buttons=[dict(label='Play',
                                                                 method='animate',
                                                                 args=[None,
                                                                       dict(frame=dict(duration=100, redraw=True),
                                                                            fromcurrent=True,
                                                                            transition=dict(duration=0, easing='linear'),
                                                                            mode='immediate'
                                                                           )
                                                                      ]
                                                                ),
                                                            dict(label='Pause',
                                                                 method='animate',
                                                                 args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate')]
                                                                )
                                                           ]
                                                  )
                                            ]
                               ),
                frames=[go.Frame(data=[go.Scatter(x=[d['epoch'] for d in data if d['regularization'] == reg_type and d['epoch'] <= k],
                                                  y=[d['accuracy'] for d in data if d['regularization'] == reg_type and d['epoch'] <= k],
                                                  mode='lines+markers',
                                                  line=dict(dash='dashdot' if reg_type == 'L1 regularization' else 'solid')
                                                 ) for reg_type in ['No regularization', 'L1 regularization', 'L2 regularization']],
                                 name=str(k+1)
                                ) for k in range(n_epochs)]
               )

# Show the plot
fig.show()


In [ ]:
from google.colab import files

# Download the GIF file
files.download('training_accuracy.gif')